In [1]:
import rospy as rp
import numpy as np

import moveit_ros_planning_interface as mrpi
import tf2_geometry_msgs
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from moveit_msgs.msg import Grasp, PlaceLocation
import moveit_commander as mc

from geometry_msgs.msg import Pose, PoseStamped
from shape_msgs.msg import SolidPrimitive
import tf_conversions

In [2]:
move_group = mc.MoveGroupCommander("arm")

In [3]:
move_group.get_active_joints()
move_group.get_end_effector_link()

'ee_link'

In [4]:
robot = mc.RobotCommander()

In [5]:
robot.get_current_state()

joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_link"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint, finger_joint, left_inner_finger_joint, left_inner_knuckle_joint,
  right_inner_knuckle_joint, right_outer_knuckle_joint, right_inner_finger_joint]
  position: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_link"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

In [6]:
def open_gripper():
    posture = JointTrajectory()
    posture.joint_names = ["finger_joint"]
    posture.points = [JointTrajectoryPoint()]
    posture.points[0].positions = [0.0]
    posture.points[0].time_from_start = rp.Duration(1)
    return posture

def close_gripper():
    posture = JointTrajectory()
    posture.joint_names = ["finger_joint"]
    posture.points = [JointTrajectoryPoint()]
    posture.points[0].positions = [0.7]
    posture.points[0].time_from_start = rp.Duration(1)
    return posture

In [7]:
def place(move_group):
    place_location = PlaceLocation()
    place_location.place_pose.header.frame_id = "base_link"
    q = tf_conversions.transformations.quaternion_from_euler(0, 0, np.pi / 2)
    place_location.place_pose.pose.orientation.x = q[0]
    place_location.place_pose.pose.orientation.y = q[1]
    place_location.place_pose.pose.orientation.z = q[2]
    place_location.place_pose.pose.orientation.w = q[3]
    place_location.place_pose.pose.position.x = 0
    place_location.place_pose.pose.position.y = 0.39
    place_location.place_pose.pose.position.z = 0.50

    # Setting pre-grasp approach
    # Defined with respect to frame_id

    place_location.pre_place_approach.direction.header.frame_id = "base_link"
    # Direction is set as positive x axis
    place_location.pre_place_approach.direction.vector.x = 1.0
    place_location.pre_place_approach.min_distance = 0.095
    place_location.pre_place_approach.desired_distance = 0.115

    # Setting post-grasp retreat
    # Defined with respect to frame_id
    place_location.post_place_retreat.direction.header.frame_id = "base_link"
    # Direction is set as positive z axis
    place_location.post_place_retreat.direction.vector.y = -1.0
    place_location.post_place_retreat.min_distance = 0.1
    place_location.post_place_retreat.desired_distance = 0.25

    # Setting posture of eef before grasp
    place_location.post_place_posture = open_gripper()

    # Set support surface as table1.
    move_group.set_support_surface_name("table2");
    # Call pick to pick up the object using the grasps given
    return move_group.place("object", place_location)

In [8]:
def pick(move_group):
    grasps = [Grasp()]
    
    grasps[0].grasp_pose.header.frame_id = "base_link"
    q = tf_conversions.transformations.quaternion_from_euler(-np.pi / 2, -np.pi / 4, -np.pi / 2)
    grasps[0].grasp_pose.pose.orientation.x = q[0]
    grasps[0].grasp_pose.pose.orientation.y = q[1]
    grasps[0].grasp_pose.pose.orientation.z = q[2]
    grasps[0].grasp_pose.pose.orientation.w = q[3]
    grasps[0].grasp_pose.pose.position.x = 0.415
    grasps[0].grasp_pose.pose.position.y = 0
    grasps[0].grasp_pose.pose.position.z = 0.5

    # Setting pre-grasp approach
    # Defined with respect to frame_id

    grasps[0].pre_grasp_approach.direction.header.frame_id = "base_link"
    # Direction is set as positive x axis
    grasps[0].pre_grasp_approach.direction.vector.x = 1.0
    grasps[0].pre_grasp_approach.min_distance = 0.095
    grasps[0].pre_grasp_approach.desired_distance = 0.115

    # Setting post-grasp retreat
    # Defined with respect to frame_id
    grasps[0].post_grasp_retreat.direction.header.frame_id = "base_link"
    # Direction is set as positive z axis
    grasps[0].post_grasp_retreat.direction.vector.z = 1.0
    grasps[0].post_grasp_retreat.min_distance = 0.1
    grasps[0].post_grasp_retreat.desired_distance = 0.25

    # Setting posture of eef before grasp
    grasps[0].pre_grasp_posture = open_gripper()

    # Setting posture of eef during grasp
    grasps[0].grasp_posture = close_gripper()

    # Set support surface as table1.
    move_group.set_support_surface_name("table1")
    # Call pick to pick up the object using the grasps given
    move_group.pick("object", grasps)

In [9]:
def add_collision_objects(planning_scene_interface):
    # Creating Environment
    # Create list to hold 3 collision objects.
    t1_pose = PoseStamped()
    t1_pose.header.frame_id = 'base_link'
    t1_pose.pose.position.x = 0
    t1_pose.pose.position.y = -0.5
    t1_pose.pose.position.z = 0.2

    # Add the first table where the cube will originally be kept.
    planning_scene_interface.add_box("table1", pose=t1_pose, size=(0.2, 0.4, 0.4))
    
    t2_pose = PoseStamped()
    t2_pose.header.frame_id = 'base_link'
    t2_pose.pose.position.x = 0
    t2_pose.pose.position.y = 0.5
    t2_pose.pose.position.z = 0.2
    # Add the second table where we will be placing the cube.
    planning_scene_interface.add_box("table2", pose=t2_pose, size=(0.2, 0.4, 0.4))

    o_pose = PoseStamped()
    o_pose.header.frame_id = 'base_link'
    
    o_pose.pose.position.x = 0
    o_pose.pose.position.y = -0.5
    o_pose.pose.position.z = 0.5
    # Define the object that we will be manipulating
    planning_scene_interface.add_box("object", pose=o_pose, size=(0.02, 0.02, 0.2))

In [10]:
rp.init_node("picker")
planning_scene_interface = mc.PlanningSceneInterface()

In [11]:
move_group.set_planning_time(45.0)

In [12]:
add_collision_objects(planning_scene_interface)

In [13]:
planning_scene_interface.get_known_object_names()

['object', 'table1', 'table2']

In [14]:
pick(move_group)

In [20]:
place(move_group)

False

In [15]:
d = rp.Duration(0.5)
for i in range(10):
    t1_pose = PoseStamped()
    t1_pose.header.frame_id = 'base_link'
    t1_pose.pose.position.x = i / 5.
    t1_pose.pose.position.y = 1.5
    t1_pose.pose.position.z = 0.2
    rp.sleep(d)
    planning_scene_interface.add_box("funbox", pose=t1_pose, size=(1, 1, 0.4))

In [16]:
planning_scene_interface.remove_world_object('funbox')

In [27]:
planning_scene_interface.remove_world_object('table1')

In [28]:
planning_scene_interface.remove_world_object('table2')